In [1]:
import os
import pandas as pd
import librosa
from pydub import AudioSegment

In [14]:
import os
from pydub import AudioSegment
from pydub.utils import make_chunks

def convert_to_wav(input_folder, output_folder):
    # Verifica se a pasta de saída existe, se não, cria-a
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Lista todos os arquivos na pasta de entrada
    for file_name in os.listdir(input_folder):
        # Verifica se o arquivo é um arquivo de áudio
        if file_name.endswith(('.mp3', '.ogg', '.flac', '.mp4', '.aac', '.m4a')):
            # Carrega o arquivo de áudio usando o pydub
            audio = AudioSegment.from_file(os.path.join(input_folder, file_name))
            
            # Gera o nome do arquivo de saída com a extensão WAV
            output_file = os.path.splitext(file_name)[0] + '.wav'
            output_path = os.path.join(output_folder, output_file)
            
            # Converte e salva o arquivo de áudio para WAV
            audio.export(output_path, format="wav")
            print(f"{file_name} convertido para {output_file}")

# Chama a função para converter os arquivos na pasta "input" e salvar na pasta "output"
convert_to_wav("\TCC\Audios pra passar", "\TCC\AudiosWAVReal")

Dia Mundial do Meio Ambiente_ Pronunciamento da Ministra Marina Silva.mp4 convertido para Dia Mundial do Meio Ambiente_ Pronunciamento da Ministra Marina Silva.wav
Discurso de Marina Silva na posse como ministra do Meio Ambiente e Mudança Climática (íntegra).mp4 convertido para Discurso de Marina Silva na posse como ministra do Meio Ambiente e Mudança Climática (íntegra).wav
Falas Negras_ Marielle Franco por Taís Araújo.mp4 convertido para Falas Negras_ Marielle Franco por Taís Araújo.wav
MARIELLE FALA SOBRE O DIREITO AO ABORTO.mp4 convertido para MARIELLE FALA SOBRE O DIREITO AO ABORTO.wav
Marielle Franco discursa na Solenidade de Posse 2018 do Cofecon.mp4 convertido para Marielle Franco discursa na Solenidade de Posse 2018 do Cofecon.wav
Marina Silva defende a demarcação de terras como atribuição do Ministério dos Povos Indígenas.mp4 convertido para Marina Silva defende a demarcação de terras como atribuição do Ministério dos Povos Indígenas.wav
marinadisc.mp4 convertido para marinad

In [15]:

# Função para extrair as características de áudio
def extrair_caracteristicas(audio, taxa_amostragem):
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=taxa_amostragem)
    chroma_mean = chroma_stft.mean(axis=1)
    chroma_over_mean = chroma_mean.mean()

    rms = librosa.feature.rms(y=audio)
    rms_mean = rms.mean()

    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=taxa_amostragem)
    spectral_centroid_mean = spectral_centroid.mean()

    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=taxa_amostragem)
    spectral_bandwidth_mean = spectral_bandwidth.mean()

    rolloff = librosa.feature.spectral_rolloff(y=audio, sr=taxa_amostragem, roll_percent=0.85)
    spectral_rolloff_mean = rolloff.mean()

    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)
    zero_crossing_rate_mean = zero_crossing_rate.mean()

    mfccs = librosa.feature.mfcc(y=audio, sr=taxa_amostragem, n_mfcc=20)
    mfccs_mean = mfccs.mean(axis=1)

    return (chroma_over_mean, rms_mean, spectral_centroid_mean, spectral_bandwidth_mean, spectral_rolloff_mean, zero_crossing_rate_mean, *mfccs_mean)

# Função para criar o DataFrame a partir dos dados
def criar_dataframe(caminho_audio):
    audio, taxa_amostragem = librosa.load(caminho_audio, sr=None)
    tamanho_janela = taxa_amostragem  # 1 segundo
    label = 'REAL'  # Substitua 'LABEL' pelo rótulo apropriado
    passo = tamanho_janela
    inicio = 0
    fim = tamanho_janela
    dados = []

    while fim <= len(audio):
        segmento = audio[inicio:fim]
        caracteristicas = extrair_caracteristicas(segmento, taxa_amostragem)
        dados.append(caracteristicas + (label,))
        inicio += passo
        fim += passo

    colunas = ['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate',
               'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10',
               'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20',
               'LABEL']
    df = pd.DataFrame(data=dados, columns=colunas)
    
    return df



# Função para processar todos os arquivos de áudio em uma pasta
def processar_pasta(pasta_audio):
    dados_totais = []

    # Percorre todos os arquivos na pasta
    for arquivo in os.listdir(pasta_audio):
        caminho_arquivo = os.path.join(pasta_audio, arquivo)
        if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.wav'):
            # Extrai o rótulo do nome do arquivo, se desejado
            label = arquivo.split('.')[0]  # Exemplo de extração de rótulo do nome do arquivo

            # Cria DataFrame para o arquivo de áudio atual
            df = criar_dataframe(caminho_arquivo)
            dados_totais.append(df)

    # Concatena todos os DataFrames em um único DataFrame final
    df_final = pd.concat(dados_totais, ignore_index=True)
    return df_final

# Pasta contendo os arquivos de áudio
pasta_audio = "\TCC\AudiosWAVReal"

# Chama a função para processar todos os arquivos de áudio na pasta
df_final = processar_pasta(pasta_audio)

# Agora você tem um único DataFrame contendo todas as características extraídas de todos os arquivos de áudio na pasta

df_final




c:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [19]:
df_final.to_csv('/TCC/DadosReal.csv', index=False)